In [1]:
import sys, os
sys.path.append(os.path.join(os.path.dirname('__file__'), '..', 'DB_and_Azure'))
import sql_db_functions as SQLf

### get data from server

In [2]:
conn, cursor = SQLf.sql_db_functions.connect_sql()

In [3]:
query = "SELECT product_characteristics.id, Brand_id , Detail, Summary, Brand FROM product_characteristics INNER JOIN Products ON product_characteristics.Brand_id = Products.Brand_Prod_id ;"
cursor.execute(query)

# Fetch the rows
rows = cursor.fetchall()
rows[0]

(10,
 50,
 '*Type: Top, Shirt  \n*Fit: Regular fit  \n*Sleeve style: Long sleeve  \n*Neckline: Crew neck  \n*Material: Cotton  \n*Formality: Casual  \n*Season: Spring, Summer, Autumn  \n*Colors: White (100%)  \n*Texture: Smooth  \n*Transparency: Opaque  \n*Details and Embellishments: Not applicable  \n*Shape: Fitted  \n*Length: Hip-length  \n*Collar Style: Crew neck  \n*Patterns: Not applicable  \n*Patterns placement: Not applicable  \n*Fluidity of fabric: Medium  \n*Fabric weight: Light  \n*Pocket Presence: No  \n*Pocket placement: Not applicable  \n*Pocket size: Not applicable  \n*Breathability: High  \n*Occasion Suitability: Casual  \n*Lapel: Not applicable',
 'The top showcased in the images is a stylish shirt featuring a relaxed fit, making it suitable for casual wear. Crafted from lightweight, breathable fabric, it is ideal for warmer seasons like spring and summer. The shirt exhibits a smooth texture with a slight sheen, enhancing its visual appeal. It is predominantly white, wh

In [4]:
conn, cursor = SQLf.sql_db_functions.connect_sql()

In [5]:
query = "SELECT Brand_id, base64 FROM product_img ;"
cursor.execute(query)

# Fetch the rows
rows_img = cursor.fetchall()
#rows_img

In [6]:
len(rows_img)

629

In [7]:
SQLf.sql_db_functions.close_connection_db(conn=conn,cursor=cursor)

### organize data

In [8]:
import pandas as pd

In [9]:
df = pd.DataFrame(rows ,columns= ['Id','prod_id','descripcion', 'Summary','Brand'])

In [10]:
df.head()

,Id,prod_id,descripcion,Summary,Brand
0,10,50,"*Type: Top, Shirt \n*Fit: Regular fit \n*Sle...",The top showcased in the images is a stylish s...,HM
1,101,101,"*Type: Top, Shirt \n*Fit: Regular fit \n*Sle...",The top showcased in the images is a fluid fab...,Mango
2,4,4,*Type: Shirt \n*Fit: Regular fit \n*Sleeve s...,"The shirt from Gucci, as shown in the images, ...",Gucci
3,2,2,*Type: Shirt \n*Fit: Regular fit \n*Sleeve s...,The showcased shirt is a Gucci Oxford cotton s...,Gucci
4,8,8,*Type: Top \n*Fit: Regular fit \n*Sleeve sty...,The top showcased in the images is a Gucci flo...,Gucci


In [11]:
df['descripcion'] = df['descripcion'].str.replace('\n*', ' / ')
df['descripcion'] = df['descripcion'].str.replace(r'\s+', ' ')
df['descripcion'] = df['descripcion'].str.replace('*', ' ')

df.head(5)

,Id,prod_id,descripcion,Summary,Brand
0,10,50,"Type: Top, Shirt / Fit: Regular fit / Sle...",The top showcased in the images is a stylish s...,HM
1,101,101,"Type: Top, Shirt / Fit: Regular fit / Sle...",The top showcased in the images is a fluid fab...,Mango
2,4,4,Type: Shirt / Fit: Regular fit / Sleeve s...,"The shirt from Gucci, as shown in the images, ...",Gucci
3,2,2,Type: Shirt / Fit: Regular fit / Sleeve s...,The showcased shirt is a Gucci Oxford cotton s...,Gucci
4,8,8,Type: Top / Fit: Regular fit / Sleeve sty...,The top showcased in the images is a Gucci flo...,Gucci


In [12]:
df['Complete_description'] = df['Summary'] + ' /' + "Brand: " + df['Brand'] + " /" + df['descripcion']
df.head()

,Id,prod_id,descripcion,Summary,Brand,Complete_description
0,10,50,"Type: Top, Shirt / Fit: Regular fit / Sle...",The top showcased in the images is a stylish s...,HM,The top showcased in the images is a stylish s...
1,101,101,"Type: Top, Shirt / Fit: Regular fit / Sle...",The top showcased in the images is a fluid fab...,Mango,The top showcased in the images is a fluid fab...
2,4,4,Type: Shirt / Fit: Regular fit / Sleeve s...,"The shirt from Gucci, as shown in the images, ...",Gucci,"The shirt from Gucci, as shown in the images, ..."
3,2,2,Type: Shirt / Fit: Regular fit / Sleeve s...,The showcased shirt is a Gucci Oxford cotton s...,Gucci,The showcased shirt is a Gucci Oxford cotton s...
4,8,8,Type: Top / Fit: Regular fit / Sleeve sty...,The top showcased in the images is a Gucci flo...,Gucci,The top showcased in the images is a Gucci flo...


In [13]:
documents = df['Complete_description'].tolist()
id = df['Id'].astype('str').tolist()
metadata = [{"source": id} for id in df['Id'].astype('str').tolist()]

In [14]:
df_image = pd.DataFrame(rows_img, columns=['prod_id', 'base64'])

In [15]:
df_image.head(2)

,prod_id,base64
0,1,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
1,1,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."


In [16]:
df_image.shape

(629, 2)

In [17]:
del rows, rows_img

### Get images 

In [18]:
import base64
from io import BytesIO
from PIL import Image

In [19]:
def base64_to_image(base64_string):
    # Remove the data URI prefix if present
    if "data:image" in base64_string:
        base64_string = base64_string.split(",")[1]

    # Decode the Base64 string into bytes
    image_bytes = base64.b64decode(base64_string)
    return image_bytes

In [20]:
def create_image_from_bytes(image_bytes):
    # Create a BytesIO object to handle the image data
    image_stream = BytesIO(image_bytes)

    # Open the image using Pillow (PIL)
    image = Image.open(image_stream)
    return image

In [21]:
def resize_image(image, new_width):
    # Open the image

    # Calculate the new height while maintaining the aspect ratio
    original_width, original_height = image.size
    aspect_ratio = original_height / original_width
    new_height = int(new_width * aspect_ratio)

    # Resize the image
    resized_image = image.resize((new_width, new_height), Image.ANTIALIAS)

    return resized_image


In [22]:
df_image.head()

,prod_id,base64
0,1,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
1,1,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
2,1,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
3,1,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."
4,2,"data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ..."


In [23]:
df_image.iloc[42]

prod_id                                                   11
base64     data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAAQ...
Name: 42, dtype: object

### Testing embeddings

In [24]:
import pandas as pd

In [25]:
import uuid
from langchain.vectorstores import Chroma
from langchain.storage import InMemoryStore
from langchain.schema.document import Document
from langchain.retrievers.multi_vector import MultiVectorRetriever

In [26]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)

In [27]:
embedding_function = SentenceTransformerEmbeddings(model_name='all-mpnet-base-v2')

c:\Users\juand\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\juand\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\juand\anaconda3\Lib\site-packages\transformers\utils\generic.py:482: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\

### organized text 

In [28]:
df = pd.merge(
            df,
            df_image.groupby('prod_id')['base64'].apply(list).reset_index(),
            on='prod_id'
        )

In [29]:
df.prod_id = df.prod_id.astype(int)
df.sort_values('prod_id',ascending=True,inplace=True)
df.reset_index(drop=True, inplace= True)
df.prod_id = df.prod_id.astype('string')
df.head(5)

,Id,prod_id,descripcion,Summary,Brand,Complete_description,base64
0,1,1,Type: Shirt / Fit: Regular fit / Sleeve s...,The Gucci shirt showcased in the images is a s...,Gucci,The Gucci shirt showcased in the images is a s...,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
1,2,2,Type: Shirt / Fit: Regular fit / Sleeve s...,The showcased shirt is a Gucci Oxford cotton s...,Gucci,The showcased shirt is a Gucci Oxford cotton s...,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
2,3,3,Type: Top / Fit: Regular fit / Sleeve sty...,The top showcased in the images is a short sil...,Gucci,The top showcased in the images is a short sil...,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
3,4,4,Type: Shirt / Fit: Regular fit / Sleeve s...,"The shirt from Gucci, as shown in the images, ...",Gucci,"The shirt from Gucci, as shown in the images, ...","[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
4,5,5,Type: Shirt / Fit: Regular fit / Sleeve s...,"The images showcase a Gucci shirt, designed wi...",Gucci,"The images showcase a Gucci shirt, designed wi...","[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."


### chunks

In [30]:
df = df[['prod_id', 'Summary','descripcion','Brand','base64']]#.append(df[['prod_id','Summary', 'descripcion','Brand','base64']])

In [32]:
df.head()

,prod_id,Summary,descripcion,Brand,base64
0,1,The Gucci shirt showcased in the images is a s...,Type: Shirt / Fit: Regular fit / Sleeve s...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
1,2,The showcased shirt is a Gucci Oxford cotton s...,Type: Shirt / Fit: Regular fit / Sleeve s...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
2,3,The top showcased in the images is a short sil...,Type: Top / Fit: Regular fit / Sleeve sty...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
3,4,"The shirt from Gucci, as shown in the images, ...",Type: Shirt / Fit: Regular fit / Sleeve s...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."
4,5,"The images showcase a Gucci shirt, designed wi...",Type: Shirt / Fit: Regular fit / Sleeve s...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA..."


In [33]:
df.Brand.unique()

array(['Gucci', 'HM', 'Prada', 'Mango'], dtype=object)

In [34]:
df['Type'] = df['Brand'].apply(lambda x: 'Luxury' if (x == 'Gucci') or (x == 'Prada') else 'Retail')

In [35]:
df.head()

,prod_id,Summary,descripcion,Brand,base64,Type
0,1,The Gucci shirt showcased in the images is a s...,Type: Shirt / Fit: Regular fit / Sleeve s...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Luxury
1,2,The showcased shirt is a Gucci Oxford cotton s...,Type: Shirt / Fit: Regular fit / Sleeve s...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Luxury
2,3,The top showcased in the images is a short sil...,Type: Top / Fit: Regular fit / Sleeve sty...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Luxury
3,4,"The shirt from Gucci, as shown in the images, ...",Type: Shirt / Fit: Regular fit / Sleeve s...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Luxury
4,5,"The images showcase a Gucci shirt, designed wi...",Type: Shirt / Fit: Regular fit / Sleeve s...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Luxury


In [36]:
persist_directory = '../../../Chromadb/chroma-retail'

In [37]:
store = InMemoryStore()
id_key = 'doc_id'

In [38]:
min(df['base64'].apply(lambda x: len(x)))

1

In [39]:
df[ df['base64'].apply(lambda x: len(x)) == 1 ]

,prod_id,Summary,descripcion,Brand,base64,Type
130,131,The top showcased in the images is a fluid fab...,"Type: Top, Shirt / Fit: Regular fit / Sle...",Mango,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Retail


In [40]:
doc_ids = df.prod_id


descripcions = [
    Document(
        page_content= df.iloc[i].descripcion, 
        metadata={
            id_key:df.iloc[i].prod_id,
            'Type': df.iloc[i].Type
            }
    )
    for i in df.index
]


In [41]:
df.head()

,prod_id,Summary,descripcion,Brand,base64,Type
0,1,The Gucci shirt showcased in the images is a s...,Type: Shirt / Fit: Regular fit / Sleeve s...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Luxury
1,2,The showcased shirt is a Gucci Oxford cotton s...,Type: Shirt / Fit: Regular fit / Sleeve s...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Luxury
2,3,The top showcased in the images is a short sil...,Type: Top / Fit: Regular fit / Sleeve sty...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Luxury
3,4,"The shirt from Gucci, as shown in the images, ...",Type: Shirt / Fit: Regular fit / Sleeve s...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Luxury
4,5,"The images showcase a Gucci shirt, designed wi...",Type: Shirt / Fit: Regular fit / Sleeve s...,Gucci,"[data:image/jpeg;base64, /9j/4AAQSkZJRgABAQAAA...",Luxury


In [42]:
final_df = pd.DataFrame()

for index, parent_row in df.iterrows(): 
    x = []
    for row in parent_row.descripcion.split('/'):
        x.append(row.split(':'))

    temp_dataframe = pd.DataFrame(x)
    temp_dataframe[0] = temp_dataframe[0].str.lower()
    temp_dataframe = temp_dataframe.sort_values(by=0)
    temp_dataframe = temp_dataframe.groupby(0)[1].apply(lambda x: ', '.join(x)).reset_index()
    temp_dataframe['prod_id'] = parent_row.prod_id
    temp_dataframe['type'] = parent_row.Type
    
        
    #temp_dataframe = temp_dataframe.set_index(0).T
    #temp_dataframe.columns = temp_dataframe.columns.str.lstrip()
    
    final_df = pd.concat([final_df,temp_dataframe], axis=0)


In [43]:
final_df

,0,1,prod_id,type
0,breathability,High,1,Luxury
1,collar style,Button-down,1,Luxury
2,colors,Light pink (100%),1,Luxury
3,details and embellishments,Buttons,1,Luxury
4,fabric weight,Light,1,Luxury
...,...,...,...,...
19,shape,Fitted,154,Retail
20,sleeve style,Long sleeve,154,Retail
21,texture,Smooth,154,Retail
22,transparency,Semi-transparent,154,Retail


In [44]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
child_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=70,)

In [45]:
doc_ids = final_df.prod_id


descripcions = [
    Document(
        page_content= final_df.iloc[i][1], 
        metadata={
            id_key:final_df.iloc[i].prod_id,
            'Type': final_df.iloc[i].type,
            'category': final_df.iloc[i][0]         
            }
    )
    for i in final_df.index
]


In [46]:
doc_ids = df.prod_id


texts = [
    Document(
        page_content= df.iloc[i].Summary, 
        metadata={
            id_key:df.iloc[i].prod_id,
            'Type': df.iloc[i].Type
            }
    )
    for i in df.index
]


In [47]:
texts[1].page_content

"The showcased shirt is a Gucci Oxford cotton shirt with a GG Shadow monogram. It has a slim fit, ideal for a tailored look, and is crafted from high-quality Oxford cotton, making it suitable for both formal and semi-formal occasions. The shirt is primarily in a light blue color, which gives it a classic, versatile appeal. The material appears to be medium-weight, offering a balance between structure and breathability, making it suitable for year-round wear. The fabric has a smooth texture with no visible transparency, ensuring it maintains a polished appearance. The shirt features a standard collar style and a button-down front, providing a traditional yet sophisticated look. The long sleeves end in buttoned cuffs, adding to its formal appeal. The GG Shadow monogram pattern is subtly placed all over the shirt, adding a touch of luxury without overwhelming the design. There are no visible pockets, lapels, or additional embellishments, keeping the design clean and elegant. The shirt's l

In [48]:
text_child = []

for i in df.index:

    child_docs=child_splitter.create_documents(
            [texts[i].page_content],
            metadatas= 
                [{
                    'doc_id':texts[i].metadata['doc_id'],
                    'Type': df.iloc[i].Type
                }]
        )
    
    for _doc in child_docs:
        text_child.append(_doc)
    

In [49]:
vectorstore = Chroma.from_documents(
                documents=text_child,
                #ids=doc_ids,
                embedding=embedding_function,
                collection_name='RAG-Child',
                #metadatas=metadata,
                persist_directory=persist_directory
            )

In [50]:
vectorstore.similarity_search_with_score('Luxury')

[(Document(page_content='blend of luxury and practicality, suitable for both professional and upscale casual settings.', metadata={'Type': 'Luxury', 'doc_id': '21'}),
  0.966240406036377),
 (Document(page_content='wear. The overall design is elegant yet relaxed, perfect for upscale beach outings or casual summer gatherings.', metadata={'Type': 'Luxury', 'doc_id': '3'}),
  1.200760841369629),
 (Document(page_content='for both professional settings and upscale social events, suitable for all seasons due to its breathable silk material.', metadata={'Type': 'Luxury', 'doc_id': '1'}),
  1.2270276546478271),
 (Document(page_content='wear to evening outings, thanks to its versatile design and luxurious fabric.', metadata={'Type': 'Retail', 'doc_id': '122'}),
  1.246087670326233)]

In [51]:
vectorstore = Chroma.from_documents(
                documents=descripcions,
                #ids=doc_ids,
                embedding=embedding_function,
                collection_name='RAG-Child-Details',
                #metadatas=metadata,
                persist_directory=persist_directory
            )

### Already loaded !!!

In [ ]:
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    docstore=store,
    id_key=id_key,
    search_kwargs={"k":5}
)

In [ ]:
docs = retriever.invoke(
    "HM"
)

In [ ]:
docs

In [ ]:
retriever.vectorstore.add_documents(texts)
#retriever.docstore.mset(list(zip(doc_ids, texts)))

In [ ]:
docs = retriever.invoke(
    "HM"
)

In [ ]:
docs

In [ ]:
df.iloc[1].Complete_description

### Talk to Chroma

In [ ]:
from apikey import apikey 
from langchain.chat_models import ChatOpenAI

In [ ]:
os.environ['OPENAI_API_KEY'] = apikey

In [ ]:
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

### lets set up the system

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
system_prompt = (
    "You are a Fashion specialist engine to sugest clothing pieces from context. "
    "If you don't know the answer to the question, say that you dont know "
    "answer concise."
    #"Explain why the selected options of the context are the best ones"
    "\n\n"
    "{context}"
)

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | turbo_llm
    | StrOutputParser()
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(turbo_llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [ ]:
query = "Im looking for clothing peaces from Mango that could be similar to Gucci clothing"

In [ ]:
response = rag_chain.invoke({"input": query})
print(response["answer"])

In [ ]:
for document in response["context"]:
    print(document)
    print()

In [ ]:
df_image[df_image.prod_id == 41]

In [ ]:
image_bytes = base64_to_image(df_image.iloc[113].base64)
img = create_image_from_bytes(image_bytes=image_bytes)
img.resize((400,500))